# Python 赋值机制

In [1]:
x = [1, 2, 3]
y = x
x[1] = 100
print y 
# 改变变量x的值，变量y的值也随着改变，
# 这与Python内部的赋值机制有关。

[1, 100, 3]


<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#简单类型" data-toc-modified-id="简单类型-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>简单类型</a></span><ul class="toc-item"><li><span><a href="#执行过程" data-toc-modified-id="执行过程-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>执行过程</a></span></li></ul></li><li><span><a href="#容器类型" data-toc-modified-id="容器类型-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>容器类型</a></span></li></ul></div>

## 简单类型

In [4]:
x = [1, 2, 3]
y = x
y[1] = 100
print x 

[1, 100, 3]


### 执行过程
```python
x = 500
y = x
y = 'foo'
```

* x = 500

**Python**分配了一个 `PyInt` 大小的内存 `pos1` 用来储存对象 `500` ，然后，Python在命名空间中让变量 `x` 指向了这一块内存，注意，整数是不可变类型，所以这块内存的内容是不可变的。

内存 | 命名空间
----- | ----
`pos1 : PyInt(500)` (不可变) | `x : pos1`

* y = x

Python并没有使用新的内存来储存变量 y 的值，而是在命名空间中，让变量 y 与变量 x 指向了**同一块内存空间**。

内存 | 命名空间
----- | ----
`pos1 : PyInt(500)` (不可变) | `x : pos1` `y : posl`

* y = 'foo'

`Python`此时分配一个 PyStr 大小的内存 `pos2` 来储存对象 `foo` ，然后改变变量 y 所指的对象。

内存 | 命名空间
----- | ----
`pos1 : PyInt(500)` (不可变) | `x : pos1`
`pos2 : PyStr('foo')` (不可变)|`y : pos2`

In [5]:
# 使用id(x)函数进行验证
x = 500
id(x)

62522492

In [6]:
y = x
id(y)

62522492

In [7]:
x is y

True

In [8]:
y = 'foo'
id(y)

55108624

In [9]:
x is y

False

Python会为每个出现的对象进行赋值，哪怕它们的值是一样的，例如：

In [10]:
x = 500
id(x)

62537272

In [11]:
y = 500
id(y)

62536792

In [12]:
x is y

False

不过，为了提高内存利用效率，对于一些**简单的对象**，如一些数值较小的int对象，Python采用了重用对象内存的办法：

In [13]:
x = 2
id(x)

51086956

In [14]:
y = 2
id(y)

51086956

In [15]:
x is y

True

## 容器类型

```python
x = [500, 501, 502]
y = x
y[1] = 600
y = [700, 800]
```

* x = [500, 501, 502]

Python为3个PyInt分配内存 `pos1 ， pos2 ， pos3` （不可变），然后为列表分配一段内存 `pos4`,它包含3个位置，分别指向这3个内存，最后再让变量 x 指向这个列表。

内存 | 命名空间
----- | ----
`pos1 : PyInt(500)` (不可变) | `x : pos4`
`pos2 : PyInt(501)` (不可变) |
`pos3 : PyInt(502)` (不可变) |
`pos4 : PyList(pos1,pos2,pos3)` (可变) | 

* y = x

并没有创建新的对象，只需要将 `y` 指向 `pos4` 即可。

内存 | 命名空间
----- | ----
`pos1 : PyInt(500)` (不可变) | `x : pos4`
`pos2 : PyInt(501)` (不可变) | `y : pos4`
`pos3 : PyInt(502)` (不可变) |
`pos4 : PyList(pos1,pos2,pos3)` (可变) | 

* y[1] = 600

之前 y[1] 这个位置指向的是 pos2 ，由于不能修改 pos2 的值，所以首先为 600 分配新内存 pos5，再把 y[1] 指向的位置修改为 pos5 。

此时，由于 pos2 位置的对象已经没有用了，Python会自动调用垃圾处理机制将它回收。


内存 | 命名空间
----- | ----
`pos1 : PyInt(500)` (不可变) | `x : pos4`
`pos2 : `垃圾回收| `y : pos4`
`pos3 : PyInt(502)` (不可变) |
`pos4 : PyList(pos1,pos5,pos3)` (可变) | 
`pos5 : PyInt(600) `(不可变)|

* y = [700, 800]

首先创建这个列表，然后将变量 y 指向它。

内存 | 命名空间
----- | ----
`pos1 : PyInt(500)` (不可变) | `x : pos4`
`pos3 : PyInt(502)` (不可变) | `y : pos8`
`pos4 : PyList(pos1,pos5,pos3)` (可变) | 
`pos5 : PyInt(600) `(不可变)|
`pos6 : PyInt(700) `(不可变)|
`pos7 : PyInt(800) `(不可变)|
`pos8 : PyList(pos6, pos7)` (可变)|

In [16]:
# 验证
x = [500, 501, 502]
print id(x[0])
print id(x[1])
print id(x[2])
print id(x)

62536792
62522492
62536888
84443048


In [17]:
y = x
print id(y)

84443048


In [18]:
x is y

True

In [22]:
y[1] = 600
print id(y) # 此时id(y)并未改变但是id(y[1])改变了

84443048


In [23]:
print id(x[1])
print id(y[1])

62536900
62536900


In [24]:
# 更改 y 的值，id(y) 的值改变
y = [700, 800]
print id(y)
print id(x)

84513536
84443048
